Requirements to run this notebook:

1. [Install `perfcapture` and its dependencies in a new virtual environment](https://github.com/zarr-developers/perfcapture).
2. Activate that venv.
3. Install additional Python dependencies within the venv: `pip install ipykernel zarr matplotlib`
4. Optionally, install `fio` to benchmark your local hard disk. (On Ubuntu: `sudo apt install fio`)

# Benchmark local hard disk using `fio`

[`fio`](https://fio.readthedocs.io/) is a disk benchmarking tool written by [Jens Axboe](https://en.wikipedia.org/wiki/Jens_Axboe) (Jens is the current Linux kernel maintainer of the block layer).





In [1]:
!mkdir -p ~/temp/fio

In [2]:
%%capture fio_json
!fio --name=read --size=1g --direct=1 --bs=64k --rw=read \
     --ioengine=libaio --iodepth=32 \
     --directory="$HOME/temp/fio/" --output-format=json

In [3]:
import json
fio_json = json.loads(fio_json.stdout)
max_gbytes_per_sec = fio_json['jobs'][0]['read']['bw_bytes'] / 1E9
print(f"This hard drive is capable of {max_gbytes_per_sec:.3f} gigabytes per second.")

This hard drive is capable of 2.049 gigabytes per second.


# Create datasets and run workloads

In [1]:
from perfcapture.workload import load_workloads_from_filename, run_workloads
from perfcapture.dataset import create_datasets_if_necessary
import pathlib
import os
import time

In [2]:
path = pathlib.Path('../recipes/simple_zarr_python_workloads.py').resolve().absolute()
os.chdir(path.parent)
workloads = load_workloads_from_filename(path)

Instantiating ZarrPythonLoadEntireArray


In [3]:
DATA_PATH = pathlib.Path("~/temp/perfcapture_data_path").expanduser()
created_at_least_1_dataset: bool = create_datasets_if_necessary(workloads, DATA_PATH)
if created_at_least_1_dataset:
    print("Waiting for data to be flushed to disk...")
    time.sleep(10)

Found 5 Dataset object(s).
LZ4_10000_Chunks already exists.
Uncompressed_1_Chunk already exists.
Uncompressed_100_Chunks already exists.
Uncompressed_10000_Chunks already exists.
LZ4_100_Chunks already exists.


In [4]:
all_results = run_workloads(workloads, keep_cache=False)

Running ZarrPythonLoadEntireArray 3 times on Uncompressed_1_Chunk!
Run 0 of 3...
Run 1 of 3...
Run 2 of 3...
  Finished!
                         mean         std
Runtime in secs     17.073791    5.995052
GB/sec to numpy      0.251496    0.073595
read_IOPS         2271.500126  590.957598
write_IOPS         607.954355  638.033437
read_GB_per_sec      0.262233    0.073946
write_GB_per_sec     0.025769    0.022747

Running ZarrPythonLoadEntireArray 3 times on LZ4_100_Chunks!
Run 0 of 3...
Run 1 of 3...
Run 2 of 3...
  Finished!
                         mean        std
Runtime in secs      3.694430   0.534608
GB/sec to numpy      1.099429   0.173316
read_IOPS         1767.177702  77.267883
write_IOPS           7.676411   6.497545
read_GB_per_sec      0.188700   0.025538
write_GB_per_sec     0.000069   0.000040

Running ZarrPythonLoadEntireArray 3 times on Uncompressed_100_Chunks!
Run 0 of 3...
Run 1 of 3...
Run 2 of 3...
  Finished!
                         mean         std
Runtime in secs

In [8]:
all_results

Runtime in secs  \
workload                  dataset                   run_ID                    
ZarrPythonLoadEntireArray Uncompressed_1_Chunk      1             23.990605   
                                                    2             13.858097   
                                                    3             13.372671   
                          LZ4_100_Chunks            1              4.040398   
                                                    2              3.964204   
                                                    3              3.078687   
                          Uncompressed_100_Chunks   1             11.566039   
                                                    2             12.210625   
                                                    3             10.723113   
                          LZ4_10000_Chunks          1              3.603084   
                                                    2              2.887366   
                                                    3              3.002183   
                          Uncompressed_10000_Chunks 1             16.909116   
                                                    2             14.805924   
                                                    3             14.743575   

                                                            GB/sec to numpy  \
workload                  dataset                   run_ID                    
ZarrPythonLoadEntireArray Uncompressed_1_Chunk      1              0.166732   
                                                    2              0.288640   
                                                    3              0.299118   
                          LZ4_100_Chunks            1              0.990001   
                                                    2              1.009030   
                                                    3              1.299255   
                          Uncompressed_100_Chunks   1              0.345840   
                                                    2              0.327584   
                                                    3              0.373026   
                          LZ4_10000_Chunks          1              1.110160   
                                                    2              1.385346   
                                                    3              1.332364   
                          Uncompressed_10000_Chunks 1              0.236559   
                                                    2              0.270162   
                                                    3              0.271305   

                                                              read_IOPS  \
workload                  dataset                   run_ID                
ZarrPythonLoadEntireArray Uncompressed_1_Chunk      1       1612.881376   
                                                    2       2446.223316   
                                                    3       2755.395687   
                          LZ4_100_Chunks            1       1799.327690   
                                                    2       1679.025600   
                                                    3       1823.179817   
                          Uncompressed_100_Chunks   1       2698.071483   
                                                    2       2737.370118   
                                                    3       3140.226164   
                          LZ4_10000_Chunks          1       2818.696428   
                                                    2       3572.460159   
                                                    3       3416.180826   
                          Uncompressed_10000_Chunks 1       2383.507216   
                                                    2       3208.648106   
                                                    3       2943.451639   

                                                             write_IOPS  \
workload             